In [1]:
import os
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
client = OpenAI()

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt="write strictly 'Yes' or 'No' to the following question: Will bitcoin exceed $80k today?\nAnswer: ",
    max_tokens=5,
    temperature=0,
    stream=False,
    logprobs=20,
)

In [40]:
from openai import OpenAI

def get_probability_from_openai(
        prompt,
        model='gpt-3.5-turbo-instruct',
        temperature=0,
        max_tokens=5,
        logprobs=20,
):
    
    client = OpenAI()
    response = client.completions.create(
        model=model,
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        stream=False,
        logprobs=logprobs,
    )
    return response

In [46]:
response = get_probability_from_openai(
    prompt="write strictly 'Yes' or 'No' to the following question: Will bitcoin exceed $80k today?\nAnswer: ",
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    max_tokens=5,
    logprobs=20,
)

In [47]:
from pprint import pprint
pprint(response)

Completion(id='cmpl-BNweF7Lc5GlesT8tlpS4LVNZSlIND', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=Logprobs(text_offset=[96], token_logprobs=[-0.055982918], tokens=['No'], top_logprobs=[{'No': -0.055982918, ' No': -3.5168955, '\n': -3.777815, '\n\n': -7.8597474, 'I': -8.266626, 'NO': -9.063478, ' \n': -9.269735, 'N': -9.302925, 'Not': -9.439436, 'Yes': -9.452101, 'Unknown': -9.518801, 'Sorry': -9.714113, "'": -9.72753, 'It': -9.912325, 'Today': -10.2064, 'no': -10.245663, 'There': -10.28842, " '": -10.295128, 'Cannot': -10.301048, 'Unfortunately': -10.305155}]), text='No')], created=1745046687, model='gpt-3.5-turbo-instruct:20230824-v2', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=1, prompt_tokens=25, total_tokens=26, completion_tokens_details=None, prompt_tokens_details=None))


In [48]:
print(response.choices[0].logprobs.top_logprobs)

[{'No': -0.055982918, ' No': -3.5168955, '\n': -3.777815, '\n\n': -7.8597474, 'I': -8.266626, 'NO': -9.063478, ' \n': -9.269735, 'N': -9.302925, 'Not': -9.439436, 'Yes': -9.452101, 'Unknown': -9.518801, 'Sorry': -9.714113, "'": -9.72753, 'It': -9.912325, 'Today': -10.2064, 'no': -10.245663, 'There': -10.28842, " '": -10.295128, 'Cannot': -10.301048, 'Unfortunately': -10.305155}]


In [ ]:
def extract_yes_no_logits_and_softmax(response):
    import numpy as np

    yes_logits = []
    no_logits = []
    yes_tokens = []
    no_tokens = []

    top_logprobs = response.choices[0].logprobs.top_logprobs[0]
    for token, logit in top_logprobs.items():
        processed_token = token.lower().strip()
        if processed_token == 'yes':
            yes_logits.append(logit)
            yes_tokens.append(token)
        elif processed_token == 'no':
            no_logits.append(logit)
            no_tokens.append(token)

    relevant_logits = np.array(yes_logits + no_logits)

    if relevant_logits.size == 0:
        return Exception("No relevant logits found for 'yes'S or 'no' tokens.")

    exp_logits = np.exp(relevant_logits - np.max(relevant_logits))
    softmax_probs = exp_logits / exp_logits.sum()

    yes_prob_sum = float(np.sum(softmax_probs[:len(yes_logits)])) if yes_logits else 0.0
    no_prob_sum = float(np.sum(softmax_probs[len(yes_logits):])) if no_logits else 0.0

    return {
        "yes_tokens_found": yes_tokens,
        "no_tokens_found": no_tokens,
        "yes_prob_sum": yes_prob_sum,
        "no_prob_sum": no_prob_sum,
        "softmax_probs_yes_no": softmax_probs.tolist()
    }

In [50]:
extract_yes_no_logits_and_softmax(response)

{'yes_tokens_found': ['Yes'],
 'no_tokens_found': ['No', ' No', 'NO', 'no'],
 'yes_prob_sum': 8.04985081849187e-05,
 'no_prob_sum': 0.999919501491815,
 'softmax_probs_yes_no': [8.04985081849187e-05,
  0.9693264563458175,
  0.03043791007391366,
  0.00011873114512202792,
  3.6403926961757085e-05]}